In [4]:
import pandas as pd

df = pd.read_csv('training_set_VU_DM.csv')



In [5]:
def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'


In [6]:

df['date_time'] = pd.to_datetime(df['date_time'])
df['time_of_day'] = df['date_time'].dt.hour.apply(get_time_of_day)
df['is_weekend'] = df['date_time'].dt.dayofweek >= 5


df['price_per_night'] = df['price_usd'] / df['srch_length_of_stay']
df['star_rating_difference'] = df['prop_starrating'] - df['visitor_hist_starrating'].fillna(df['prop_starrating'])
df['is_domestic'] = (df['prop_country_id'] == df['visitor_location_country_id']).astype(int)

print(df[['time_of_day', 'star_rating_difference', 'is_domestic']].head())



  time_of_day  star_rating_difference  is_domestic
0     Morning                     0.0            0
1     Morning                     0.0            0
2     Morning                     0.0            0
3     Morning                     0.0            0
4     Morning                     0.0            0


In [ ]:
#Competition features

df[[f'comp{i}_rate' for i in range(1, 9)]].fillna(0, inplace=True)

df['rate_advantage_count'] = (df[[f'comp{i}_rate' for i in range(1, 3)]].fillna(0) > 0).sum(axis=1)
df['rate_disadvantage_count'] = (df[[f'comp{i}_rate' for i in range(1, 3)]].fillna(0) < 0).sum(axis=1)
df['availability_advantage_count'] = (df[[f'comp{i}_inv' for i in range(1, 3)]].fillna(0) > 0).sum(axis=1)
df['avg_rate_percent_diff'] = df[[f'comp{i}_rate_percent_diff' for i in range(1, 3)]].mean(axis=1, skipna=True)
df['max_rate_percent_diff'] = df[[f'comp{i}_rate_percent_diff' for i in range(1, 3)]].max(axis=1, skipna=True)
df['min_rate_percent_diff'] = df[[f'comp{i}_rate_percent_diff' for i in range(1, 3)]].min(axis=1, skipna=True)

df.head

In [7]:
df.to_csv('feature_eng_out.csv', index=False)